In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import os
import keras
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Input, Dense
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf
import datetime


2024-02-28 15:53:29.273124: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 15:53:29.273197: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 15:53:29.273226: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 15:53:29.279833: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 15:53:30.035906: W tensorflow/compiler/

In [208]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("solution_submission.csv")

In [209]:
train.isnull().any()

Season      False
DateTime    False
HomeTeam    False
AwayTeam    False
Referee     False
HS          False
AS          False
HST         False
AST         False
HC          False
AC          False
HF          False
AF          False
HY          False
AY          False
HR          False
AR          False
HTHG        False
HTAG        False
HTR         False
FTHG        False
FTAG        False
FTR         False
dtype: bool

In [207]:
unique_teams = pd.unique(train[['HomeTeam', 'AwayTeam']].values.ravel())
unique_teams = unique_teams.tolist()
len(unique_teams)

43

In [210]:
train = train.drop(['FTHG', "FTAG", "Season"], axis = 1)


In [211]:
test = test.drop(["FTR", "ID", "Season"],axis=1)

In [212]:
train = train.drop(['Referee'], axis= 1)

In [213]:
test = test.drop(['Referee'], axis=1)

In [195]:
train.head()

,DateTime,HomeTeam,AwayTeam,Referee,HS,AS,HST,AST,HY,HR,HTHG,HTAG,HTR,FTR
0,2000-08-19T00:00:00Z,Charlton,Man City,Rob Harris,17,8,14,4,1,0,2,0,H,H
1,2000-08-19T00:00:00Z,Chelsea,West Ham,Graham Barber,17,12,10,5,1,0,1,0,H,H
2,2000-08-19T00:00:00Z,Coventry,Middlesbrough,Barry Knight,6,16,3,9,5,1,1,1,D,A
3,2000-08-19T00:00:00Z,Derby,Southampton,Andy D'Urso,6,13,4,6,1,0,1,2,A,D
4,2000-08-19T00:00:00Z,Leeds,Everton,Dermot Gallagher,17,12,8,6,1,0,2,0,H,H


In [74]:
test.head()

,HomeTeam,AwayTeam,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTHG,HTAG,HTR
0,Fulham,Arsenal,C Kavanagh,5,13,2,6,2,3,12,12,2,2,0,0,0,1,A
1,Crystal Palace,Southampton,J Moss,5,9,3,5,7,3,14,11,2,1,0,0,1,0,H
2,Liverpool,Leeds,M Oliver,22,6,6,3,9,0,9,6,1,0,0,0,3,2,H
3,West Ham,Newcastle,S Attwell,15,15,3,2,8,7,13,7,2,2,0,0,0,0,D
4,West Brom,Leicester,A Taylor,7,13,1,7,2,5,12,9,1,1,0,0,0,0,D


In [214]:
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [215]:
train_numerics = train.select_dtypes(include = ['float64', 'int64']).drop(['HomeTeam', 'AwayTeam', "Referee", "HTR", "FTR"], axis=1, errors= 'ignore')
test_numerics = test.select_dtypes(include = ['float64', 'int64']).drop(['HomeTeam', 'AwayTeam', "Referee", "HTR"], axis=1, errors= 'ignore')

In [216]:
standrdized_train_data = standard_scaler.fit_transform(train_numerics)
standrdized_test_data = standard_scaler.fit_transform(test_numerics)

In [217]:
standrdized_train_df = pd.DataFrame(standrdized_train_data, columns=train_numerics.columns)
standrdized_test_df = pd.DataFrame(standrdized_test_data, columns=test_numerics.columns)

In [219]:
standrdized_train_df['HomeTeam'] = train["HomeTeam"]
standrdized_train_df['AwayTeam'] = train["AwayTeam"]

standrdized_train_df['HTR'] = train['HTR']
standrdized_train_df['FTR'] = train["FTR"]

standrdized_test_df['HomeTeam'] = test["HomeTeam"]
standrdized_test_df['AwayTeam'] = test["AwayTeam"]

standrdized_test_df['HTR'] = test['HTR']

In [220]:
standrdized_train_df

,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTHG,HTAG,HomeTeam,AwayTeam,HTR,FTR
0,0.664129,-0.566731,2.311087,-0.294172,-0.041677,0.444008,0.411162,-0.000963,-0.339952,0.188338,-0.249877,-0.304949,1.583839,-0.698127,Charlton,Man City,H,H
1,0.664129,0.316426,1.117437,0.061040,0.280875,0.810240,1.988693,0.503545,-0.339952,0.188338,-0.249877,-0.304949,0.381684,-0.698127,Chelsea,West Ham,H,H
2,-1.438946,1.199583,-0.971451,1.481888,0.603426,-0.288456,0.937005,2.269321,3.027447,0.974374,3.649646,-0.304949,0.381684,0.701072,Coventry,Middlesbrough,D,A
3,-1.438946,0.537215,-0.673038,0.416252,-0.364229,1.176472,-0.114682,0.251291,-0.339952,-0.597697,-0.249877,-0.304949,0.381684,2.100271,Derby,Southampton,A,D
4,0.664129,0.316426,0.520612,0.416252,-0.041677,-0.288456,2.514537,2.017067,-0.339952,0.974374,-0.249877,-0.304949,1.583839,-0.698127,Leeds,Everton,H,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,0.090563,-0.787521,-0.971451,-0.649383,-1.009332,-0.654688,0.148240,-0.253216,-0.339952,1.760410,3.649646,-0.304949,-0.820471,-0.698127,Leicester,Man United,D,A
7596,3.340770,-1.229099,1.117437,-0.294172,0.925978,-1.753383,-1.166370,-2.018992,-0.339952,-0.597697,-0.249877,-0.304949,1.583839,-0.698127,Man City,Norwich,H,H
7597,-2.012512,0.758005,-1.269863,0.416252,-1.331884,-0.288456,-0.114682,-1.766738,-0.339952,-1.383733,-0.249877,-0.304949,0.381684,0.701072,Newcastle,Liverpool,D,A
7598,-0.100626,-1.229099,-0.673038,-0.649383,0.925978,-1.387151,-0.640526,1.008052,-1.181802,-0.597697,-0.249877,-0.304949,-0.820471,0.701072,Southampton,Sheffield United,A,H


In [221]:
train.dtypes

DateTime    object
HomeTeam    object
AwayTeam    object
HS           int64
AS           int64
HST          int64
AST          int64
HC           int64
AC           int64
HF           int64
AF           int64
HY           int64
AY           int64
HR           int64
AR           int64
HTHG         int64
HTAG         int64
HTR         object
FTR         object
dtype: object

In [222]:
print(train.columns, test.columns)

Index(['DateTime', 'HomeTeam', 'AwayTeam', 'HS', 'AS', 'HST', 'AST', 'HC',
       'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTHG', 'HTAG', 'HTR', 'FTR'],
      dtype='object') Index(['DateTime', 'HomeTeam', 'AwayTeam', 'HS', 'AS', 'HST', 'AST', 'HC',
       'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTHG', 'HTAG', 'HTR'],
      dtype='object')


In [223]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(standrdized_train_df, label="FTR")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(standrdized_test_df)


In [224]:
tuner = tfdf.tuner.RandomSearch(
    num_trials=100, 
    use_predefined_hps = True,
    trial_num_threads = 1,
    trial_maximum_training_duration_seconds= None
)

rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION, hyperparameter_template="benchmark_rank1")
rf.compile(metrics=["accuracy"])
rf_his =rf.fit(x = train_ds, verbose=2)

print(rf.summary())

Resolve hyper-parameter template "benchmark_rank1" to "benchmark_rank1@v1" -> {'winner_take_all': True, 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}.
Use /tmp/tmp7i870f75 as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'HS': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'AS': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'HST': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'AST': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'HC': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'AC': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'HF': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'AF': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'HY': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'AY': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'HR': <tf.Tensor 'data_10:0' shape=(Non

[INFO 24-02-28 18:00:32.4456 KST kernel.cc:773] Start Yggdrasil model training
[INFO 24-02-28 18:00:32.4456 KST kernel.cc:774] Collect training examples
[INFO 24-02-28 18:00:32.4456 KST kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-02-28 18:00:32.4457 KST kernel.cc:393] Number of batches: 8
[INFO 24-02-28 18:00:32.4457 KST kernel.cc:394] Number of examples: 7600
[INFO 24-02-28 18:00:32.4477 KST kernel.cc:794] Training dataset:
Number of records: 7600
Number of columns: 18

Number of columns by type:
	NUMERICAL: 14 (77.7778%)
	CATEGORICAL: 4 (22.2222%)

Columns:

NUMERICAL: 14 (77.7778%)
	0: "AC" NUMERICAL mean:1.63805e-08 min:-1.75338 max:5.2

Model trained in 0:00:07.899597
Compiling model...
Model compiled.
Model: "random_forest_model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (17):
	AC
	AF
	AR
	AS
	AST
	AY
	AwayTeam
	HC
	HF
	HR
	HS
	HST
	HTAG
	HTHG
	HTR
	HY
	HomeTeam

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.      "HTR"  0.202950 ################
    2.      "AST"  0.174292 ############
    3.       "AS"  0.126181 #####
    4. "AwayTeam"  0.124761 #####
    5. "HomeTeam"  0.124441 #####
    6.     "HTHG"  0.123002 #####
    7.     "HTAG"  0.120678 ####
    8.       "AC"  0.119247 ####
    9.      "HST"  0.118893 ####
   10.       "AF"  0.118397 ####
   11.       "AR"  0.118210 ##

In [225]:
predictions = rf.predict(test_ds)


        


1/1 [==============================] - 0s 120ms/step


In [226]:
predictions

array([[0.836666  , 0.13666661, 0.02666667],
       [0.08333332, 0.27999982, 0.6366662 ],
       [0.00666667, 0.1033333 , 0.8899993 ],
       ...,
       [0.04      , 0.10666663, 0.85333264],
       [0.04      , 0.15999992, 0.79999936],
       [0.04333334, 0.06666667, 0.8899993 ]], dtype=float32)

In [227]:
print(predictions)

[[0.836666   0.13666661 0.02666667]
 [0.08333332 0.27999982 0.6366662 ]
 [0.00666667 0.1033333  0.8899993 ]
 ...
 [0.04       0.10666663 0.85333264]
 [0.04       0.15999992 0.79999936]
 [0.04333334 0.06666667 0.8899993 ]]


In [228]:
a = []
for i in predictions :
    i = i.tolist()
    if i.index(max(i)) == 2 :
        a.append('H')
    elif i.index(max(i))==1:
        a.append('D')
    elif i.index(max(i))==0:
        a.append('A')

In [229]:
print(a)
len(a)

['A', 'H', 'H', 'D', 'A', 'D', 'A', 'A', 'H', 'H', 'A', 'H', 'D', 'A', 'A', 'H', 'D', 'A', 'D', 'A', 'H', 'A', 'A', 'H', 'H', 'H', 'A', 'H', 'H', 'H', 'A', 'H', 'A', 'H', 'H', 'D', 'A', 'H', 'A', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'D', 'D', 'A', 'H', 'A', 'D', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'A', 'A', 'H', 'A', 'A', 'D', 'H', 'H', 'A', 'H', 'H', 'A', 'H', 'H', 'H', 'A', 'H', 'D', 'A', 'A', 'A', 'H', 'H', 'A', 'D', 'H', 'H', 'H', 'H', 'A', 'H', 'H', 'H', 'H', 'H', 'H', 'A', 'A', 'A', 'A', 'H', 'H', 'H', 'A', 'A', 'H', 'H', 'D', 'D', 'D', 'H', 'D', 'H', 'H', 'A', 'H', 'H', 'H', 'D', 'H', 'A', 'H', 'A', 'H', 'H', 'A', 'D', 'A', 'A', 'A', 'H', 'A', 'D', 'A', 'H', 'A', 'H', 'H', 'D', 'H', 'D', 'H', 'H', 'A', 'H', 'A', 'H', 'A', 'A', 'H', 'D', 'H', 'D', 'A', 'H', 'A', 'A', 'H', 'H', 'H', 'A', 'A', 'D', 'A', 'H', 'H', 'A', 'D', 'H', 'H', 'H', 'H', 'A', 'H', 'A', 'H', 'A', 'A', 'H', 'H', 'H', 'H', 'D', 'A', 'D', 'H', 'A', 'A', 'A', 'A', 'A', 'H', 'D', 'H', 'A', 'A', 'D', 'D', 'H', 'A', 'D',

689

In [230]:
sub['FTR'] = a
sub.to_csv('submission.csv', index=False)
sub.head()

,ID,FTR
0,1,A
1,2,H
2,3,H
3,4,D
4,5,A


In [231]:
sub

,ID,FTR
0,1,A
1,2,H
2,3,H
3,4,D
4,5,A
...,...,...
684,685,A
685,686,H
686,687,H
687,688,H
